In [4]:
import os
import re

import numpy as np
import us
from tqdm import tqdm_notebook
import pandas as pd
# Census API wrapper package:
#       https://github.com/datamade/census
from census import Census, CensusException

In [5]:
#os.chdir("/media/wkg/storage/mcbi-datapalooza-2019")
os.chdir("/Users/wigasper/Documents/mcbi-datapalooza-2019")

In [3]:
zip_data = pd.read_csv("zipcodes.csv", index_col=None)

# Remove pesky "Unnamed" column
zip_data = zip_data.loc[:, ~zip_data.columns.str.contains('Unnamed')]

# Change zips to str and pad with 0s
zip_data["zip"] = zip_data["zip"].apply(lambda x: str(x).zfill(5))

# Create Census object with API key
cens = Census("641afb80c092a21ba85b039d816e211551bccad4")

zip_data contains basic geographic zip code data for 44,336 zip codes in the US.

In [4]:
zip_data.head()

,zip,city,state,latitude,longitude
0,00210,Portsmouth,NH,43.005895,-71.013202
1,00211,Portsmouth,NH,43.005895,-71.013202
2,00212,Portsmouth,NH,43.005895,-71.013202
3,00213,Portsmouth,NH,43.005895,-71.013202
4,00214,Portsmouth,NH,43.005895,-71.013202


This function, get_census_val() is used to retrieve data from the Census's 5-year American Community Survey, which provides accurate estimates for a huge number of variables for US ZIP codes. We are using this data for a number of features in our model.

In [6]:
# Still needs to be tested with every variable change
def get_census_val(cens_obj, variable, zipcode):
    try:
        result = cens_obj.acs5.zipcode(variable, zipcode)
        if len(result) > 0:
            return result[0].get(variable)
        else:
            return 0.0
    except ConnectionError:
        return None
    except CensusException:
        return None

# Population Demographics
We want to find out the total population for each zip code and the populations of the target markets. To do this, we use the U.S. Census API by way of the Census package for Python.

First, we get total population values for each zip code.

In [ ]:
# Put zip codes into a list for ease of processing
zips = [[zipcode, None] for zipcode in zip_data["zip"]]

# Get populations for zip codes if value is None. I did it this way to be able
# non-redundantly call the API in batches in case of the common ConnectionError
for zipcode in tqdm_notebook(zips):
    if zipcode[1] is None:
        zipcode[1] = get_census_val(cens, "B01003_001E", zipcode[0])
        
pops = pd.DataFrame(zips, columns=["zip", "population"])

zip_data = pd.merge(zip_data, pops, how="inner", on="zip")

# Save, because the API pulls take a long time
zip_data.to_csv("zip_data.csv")

Next, we request population data from the API for the age range that we need (45-85).

In [ ]:
populated_zips = zip_data[zip_data["population"] > 0]

zips = [[zipcode, None] for zipcode in populated_zips["zip"]]
for zipcode in tqdm_notebook(zips):
    if zipcode[1] is None:
        zipcode[1] = get_census_val(cens, "B07001_010E", zipcode[0])       
pops = pd.DataFrame(zips, columns=["zip", "pop_45-49"])
zip_data = pd.merge(zip_data, pops, how="left", on="zip")

zips = [[zipcode, None] for zipcode in populated_zips["zip"]]
for zipcode in tqdm_notebook(zips):
    if zipcode[1] is None:
        zipcode[1] = get_census_val(cens, "B07001_011E", zipcode[0])      
pops = pd.DataFrame(zips, columns=["zip", "pop_50-54"])
zip_data = pd.merge(zip_data, pops, how="left", on="zip")

zips = [[zipcode, None] for zipcode in populated_zips["zip"]]
for zipcode in tqdm_notebook(zips):
    if zipcode[1] is None:
        zipcode[1] = get_census_val(cens, "B07001_012E", zipcode[0])
pops = pd.DataFrame(zips, columns=["zip", "pop_55-59"])
zip_data = pd.merge(zip_data, pops, how="left", on="zip")

zips = [[zipcode, None] for zipcode in populated_zips["zip"]]
for zipcode in tqdm_notebook(zips):
    if zipcode[1] is None:
        zipcode[1] = get_census_val(cens, "B07001_013E", zipcode[0])
pops = pd.DataFrame(zips, columns=["zip", "pop_60-64"])
zip_data = pd.merge(zip_data, pops, how="left", on="zip")

zips = [[zipcode, None] for zipcode in populated_zips["zip"]]
for zipcode in tqdm_notebook(zips):
    if zipcode[1] is None:
        zipcode[1] = get_census_val(cens, "B07001_014E", zipcode[0])
pops = pd.DataFrame(zips, columns=["zip", "pop_65-69"])
zip_data = pd.merge(zip_data, pops, how="left", on="zip")

zips = [[zipcode, None] for zipcode in populated_zips["zip"]]
for zipcode in tqdm_notebook(zips):
    if zipcode[1] is None:
        zipcode[1] = get_census_val(cens, "B07001_015E", zipcode[0])
pops = pd.DataFrame(zips, columns=["zip", "pop_70-74"])
zip_data = pd.merge(zip_data, pops, how="left", on="zip")

zips = [[zipcode, None] for zipcode in populated_zips["zip"]]
for zipcode in tqdm_notebook(zips):
    if zipcode[1] is None:
        zipcode[1] = get_census_val(cens, "B07001_016E", zipcode[0])
pops = pd.DataFrame(zips, columns=["zip", "pop_75-inf"])
zip_data = pd.merge(zip_data, pops, how="left", on="zip")

# Save, because the API pulls take a long time
zip_data.to_csv("zip_data.csv")

We also obtained the median individual income for each zip code.

In [ ]:
zips = [[zipcode, None] for zipcode in populated_zips["zip"]]
for zipcode in tqdm_notebook(zips):
    if zipcode[1] is None:
        zipcode[1] = get_census_val(cens, "B19326_002E", zipcode[0])
incs = pd.DataFrame(zips, columns=["zip", "median_indiv_income"])
zip_data = pd.merge(zip_data, incs, how="left", on="zip")

# Save, because the API pulls take a long time
zip_data.to_csv("zip_data.csv")

This next cell is just a load point to be used when reproducing results or when adding data after the last API pull was completed

In [6]:
# Read in 
zip_data = pd.read_csv("zip_data.csv", index_col=None)

# remove pesky "Unnamed" column
zip_data = zip_data.loc[:, ~zip_data.columns.str.contains('Unnamed')]

# Change zips to str and pad with 0s
zip_data["zip"] = zip_data["zip"].apply(lambda x: str(x).zfill(5))

We obtained life expectancy data by Census tract from the [CDC](https://www.cdc.gov/nchs/nvss/usaleep/usaleep.html) and roughly converted these values to their zip codes with [HUD data](https://www.huduser.gov/portal/datasets/usps_crosswalk.html). Converting arbitrarily-defined geographic areas is not perfect - there are many single zip codes containing multiple census tracts as well as single census tracts containing multiple zip codes.

In [7]:
# read in
life_exp = pd.read_csv("life_expect_by_census_tract.csv", index_col=None)

# Drop unneeded columns
life_exp = pd.DataFrame(life_exp, columns=['Tract ID', 'e(0)'])
life_exp = life_exp.rename(index=str, columns={"Tract ID": "tract",
                                               "e(0)": "life_expectancy"})

# Convert Census tracts to zip codes
# Read in
zip_to_tract = pd.read_csv("zip_to_tract.csv", index_col=None)

# Change zips to str and pad with 0s
zip_to_tract["zip"] = zip_to_tract["zip"].apply(lambda x: str(x).zfill(5))
zip_to_tract = pd.DataFrame(zip_to_tract, columns=['zip', 'tract'])

# Merge and drop duplicates
life_exp = pd.merge(life_exp, zip_to_tract, how="left", on="tract")
life_exp = life_exp.drop_duplicates("zip")
life_exp = pd.DataFrame(life_exp, columns=['zip', 'life_expectancy'])

# Merge with zip_data
zip_data = pd.merge(zip_data, life_exp, how="left", on="zip")

# Income Tax Statistics
The files 16zpallagi.csv and 16zpallnoagi.csv, used below, were obtained from the [IRS's Individual Income Tax Statistics](https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2016-zip-code-data-soi) organized by zip code for 2016.

Using this data, we add the count of returns in the provided income ranges for every ZIP code to the zip_data dataframe:

In [8]:
income = pd.read_csv("16zpallagi.csv", index_col=None)

agis = pd.DataFrame(income, columns=["zipcode", "agi_stub", "N1"])

agis = pd.pivot_table(agis, values="N1", index="zipcode", columns="agi_stub")
agis = agis.reset_index()

agis["zipcode"] = agis["zipcode"].apply(lambda x: str(x).zfill(5))

agis = agis.rename(index=str, columns={"zipcode": "zip",
                                       1: "num_tax_returns_0-25k",
                                       2: "num_tax_returns_25k-50k",
                                       3: "num_tax_returns_50k-75k",
                                       4: "num_tax_returns_75k-100k",
                                       5: "num_tax_returns_100k-200k",
                                       6: "num_tax_returns_200k-inf"})

agis = agis[1:]

zip_data = pd.merge(zip_data, agis, how="left", on="zip")

Here we add the number of elderly tax returns for each zip code:

In [9]:
income_noagi = pd.read_csv("16zpallnoagi.csv", index_col=None)

income_noagi = pd.DataFrame(income_noagi, columns=["ZIPCODE", "ELDERLY"])
income_noagi = income_noagi.rename(index=str, columns={"ZIPCODE": "zip",
                                                       "ELDERLY": "num_elderly_tax_returns"})
income_noagi["zip"] = income_noagi["zip"].apply(lambda x: str(x).zfill(5))

zip_data = pd.merge(zip_data, income_noagi, how="left", on="zip")

Some random samples to see what the dataframe looks like after adding the population, median income, and income tax information:

In [10]:
zip_data.sample(n=10, random_state=2)

,zip,city,state,latitude,longitude,population,pop_45-49,pop_50-54,pop_55-59,pop_60-64,...,pop_75-inf,median_indiv_income,life_expectancy,num_tax_returns_0-25k,num_tax_returns_25k-50k,num_tax_returns_50k-75k,num_tax_returns_75k-100k,num_tax_returns_100k-200k,num_tax_returns_200k-inf,num_elderly_tax_returns
28530,62081,Rockbridge,IL,39.261499,-90.23039,386.0,37.0,31.0,6.0,16.0,...,26.0,39000.0,83.2,60.0,60.0,0.0,30.0,0.0,0.0,40.0
27866,60683,Chicago,IL,41.811929,-87.68732,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37099,80755,Vernon,CO,39.940858,-102.36417,162.0,17.0,4.0,4.0,13.0,...,1.0,-666666666.0,78.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27619,60192,Schaumburg,IL,42.065827,-88.21399,16026.0,1266.0,1276.0,1565.0,1383.0,...,523.0,75594.0,85.2,1860.0,1090.0,830.0,830.0,2240.0,840.0,1940.0
34970,76622,Aquilla,TX,31.824495,-97.23306,1110.0,55.0,101.0,117.0,62.0,...,65.0,31250.0,74.9,160.0,140.0,70.0,50.0,50.0,0.0,130.0
43149,97525,Gold Hill,OR,42.432422,-123.08639,4435.0,227.0,390.0,480.0,523.0,...,367.0,32917.0,80.2,900.0,600.0,340.0,210.0,220.0,60.0,840.0
8128,19025,Dresher,PA,40.149164,-75.16047,5426.0,459.0,430.0,369.0,462.0,...,506.0,81964.0,81.3,750.0,370.0,300.0,250.0,720.0,600.0,1010.0
22395,48853,Maple Rapids,MI,43.102399,-84.69278,664.0,53.0,70.0,45.0,35.0,...,29.0,31154.0,79.3,140.0,80.0,40.0,30.0,0.0,0.0,70.0
13705,30370,Atlanta,GA,33.844371,-84.47405,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38435,85139,Maricopa,AZ,32.980000,-112.11000,18327.0,1216.0,1091.0,1063.0,1051.0,...,671.0,32396.0,78.1,2410.0,2190.0,1130.0,680.0,630.0,60.0,1400.0


There are lots of ZIP codes with 0 population and no tax returns. ZIP codes are not really meaningful representations of geographical areas. ZIP codes designate postal delivery points, and as such, there are *lots* of delivery points in the U.S. that can't represent a geographic area containing permanent inhabitants. Some examples of these delivery points are: governmental agencies, universities, businesses, or simply any building that receives a huge amount of mail.

For some reason, the API returned median income values of -6... for a number of smaller zip codes. These will have to be cleaned up prior to usage in our model.

# Regional Price Parity Values
Next, we add the Regional Price Parity for our ZIP codes. Regional Price Parity is a relative metric to describe the cost of goods, rent, and services in comparison to the national average. For more information and the source of our data, see the [Bureau of Economic Analysis's website](https://www.bea.gov/data/prices-inflation/regional-price-parities-state-and-metro-area).

The RPP data was available for a limited number of CBSAs (core-based statistical area), and we roughly convert these to zip codes using [HUD data](https://www.huduser.gov/portal/datasets/usps_crosswalk.html). For zipcodes with RPP data, we used the obtained data. For ZIP codes without RPP data, we used the state-level RPP data.

In [11]:
# Read in RPP CBSA-specific data and clean up
rpp = pd.read_csv("RegionalPriceParities.csv")
rpp = rpp[rpp["LineCode"]==1]
rpp = pd.DataFrame(rpp, columns=["GeoFips", "2016"])
rpp = rpp.rename(index=str, columns={"GeoFips": "cbsa", "2016": "rpp"})

# cbsa_to_zip to convert between CBSA and zip codes
cbsa_to_zip = pd.read_csv("cbsa_to_zip.csv")
cbsa_to_zip["zip"] = cbsa_to_zip["zip"].apply(lambda x: str(x).zfill(5))
cbsa_to_zip = pd.DataFrame(cbsa_to_zip, columns=["zip", "cbsa"])

# merge CBSAs to zip codes, drop duplicates due to redundant CBSAs (probably due to 
# abritrary geographic decisions, some zip codes have multiple CBSAs)
zip_data = pd.merge(zip_data, cbsa_to_zip, how="left", on="zip")
zip_data = zip_data.drop_duplicates("zip")

# merge in RPP data
zip_data = pd.merge(zip_data, rpp, how="left", on="cbsa")
# drop CBSA column
zip_data = zip_data.loc[:, ~zip_data.columns.str.contains("cbsa")]

We still need to get the state-level data for ZIP codes without metro-area RPP data. In this case, we downloaded the portioned state RPP data and used the non-metropolitan values because we have already captured data for major metropolitan areas and we want to get more accurate values for non-metro areas (which were significantly lower than metro area RPP values).

In [12]:
# read in
rpp_state = pd.read_csv("RPP_by_state_w_portions.csv", index_col=None)

# Drop Metropolitan data
rpp_state = rpp_state.loc[rpp_state.GeoName.str.contains("Nonmetropolitan"), :]
rpp_state = rpp_state[rpp_state["LineCode"]==1]

# Clean up state names
pat = r"^(.*)\s\(Nonmetropolitan Portion\)"
rpp_state["GeoName"] = rpp_state["GeoName"].apply(lambda x: re.sub(pat, r"\1", x))

# Need to convert names to two letter codes
states_dict = us.states.mapping("name", "abbr")
rpp_state["GeoName"] = rpp_state["GeoName"].apply(lambda x: states_dict[x])
rpp_state = pd.DataFrame(rpp_state, columns=["GeoName", "2016"])

# Some states don't have nonmetro areas, setting these values manually to the metro area values
rpp_state.loc[[64], ["2016"]] = 100.3
rpp_state.loc[[72], ["2016"]] = 116.4
rpp_state.loc[[248], ["2016"]] = 113.5
rpp_state.loc[[320], ["2016"]] = 99.8

# Change rpp_state to dict to allow us to easily replace NAs in the DF with the state
# non-metro RPP 
rpp_state = rpp_state.set_index("GeoName").T.to_dict("list")

# Fill NAs with their state abbreviation
zip_data.rpp = zip_data.rpp.fillna(zip_data.state)

# Get RPP function
# Above, we filled NAs for the RPP variable with the containing state abbreviation, this function
# replaces that state abbreviation with the non-metro RPP for the state
def get_rpp_for_state(rpp):
    if re.match("[^\d]{2}", rpp):
        if rpp in rpp_state.keys():
            return rpp_state[rpp][0]
        else:
            return np.nan
    else:
        return rpp

zip_data["rpp"] = zip_data["rpp"].apply(lambda x: get_rpp_for_state(x))

Now, after adding the RPP data:

In [13]:
zip_data.sample(n=10, random_state=2)

,zip,city,state,latitude,longitude,population,pop_45-49,pop_50-54,pop_55-59,pop_60-64,...,median_indiv_income,life_expectancy,num_tax_returns_0-25k,num_tax_returns_25k-50k,num_tax_returns_50k-75k,num_tax_returns_75k-100k,num_tax_returns_100k-200k,num_tax_returns_200k-inf,num_elderly_tax_returns,rpp
28530,62081,Rockbridge,IL,39.261499,-90.23039,386.0,37.0,31.0,6.0,16.0,...,39000.0,83.2,60.0,60.0,0.0,30.0,0.0,0.0,40.0,84.6
27866,60683,Chicago,IL,41.811929,-87.68732,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84.6
37099,80755,Vernon,CO,39.940858,-102.36417,162.0,17.0,4.0,4.0,13.0,...,-666666666.0,78.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.4
27619,60192,Schaumburg,IL,42.065827,-88.21399,16026.0,1266.0,1276.0,1565.0,1383.0,...,75594.0,85.2,1860.0,1090.0,830.0,830.0,2240.0,840.0,1940.0,103.8
34970,76622,Aquilla,TX,31.824495,-97.23306,1110.0,55.0,101.0,117.0,62.0,...,31250.0,74.9,160.0,140.0,70.0,50.0,50.0,0.0,130.0,87.3
43149,97525,Gold Hill,OR,42.432422,-123.08639,4435.0,227.0,390.0,480.0,523.0,...,32917.0,80.2,900.0,600.0,340.0,210.0,220.0,60.0,840.0,97.6
8128,19025,Dresher,PA,40.149164,-75.16047,5426.0,459.0,430.0,369.0,462.0,...,81964.0,81.3,750.0,370.0,300.0,250.0,720.0,600.0,1010.0,105.9
22395,48853,Maple Rapids,MI,43.102399,-84.69278,664.0,53.0,70.0,45.0,35.0,...,31154.0,79.3,140.0,80.0,40.0,30.0,0.0,0.0,70.0,93.0
13705,30370,Atlanta,GA,33.844371,-84.47405,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.7
38435,85139,Maricopa,AZ,32.980000,-112.11000,18327.0,1216.0,1091.0,1063.0,1051.0,...,32396.0,78.1,2410.0,2190.0,1130.0,680.0,630.0,60.0,1400.0,97.1


# Cleanup and Feature Engineering
We still need to do some cleanup and some minor feature engineering. It is probably good to know the frequency of age groups in populations, in addition to the total counts. We also need to fix the extreme negative values for median individual income that the Census API sometimes returned. 

In [14]:
# Get relative age group frequencies
zip_data["freq_pop_45-49"] = zip_data["pop_45-49"] / zip_data["population"]
zip_data["freq_pop_50-54"] = zip_data["pop_50-54"] / zip_data["population"]
zip_data["freq_pop_55-59"] = zip_data["pop_55-59"] / zip_data["population"]
zip_data["freq_pop_60-64"] = zip_data["pop_60-64"] / zip_data["population"]
zip_data["freq_pop_65-69"] = zip_data["pop_65-69"] / zip_data["population"]
zip_data["freq_pop_70-74"] = zip_data["pop_70-74"] / zip_data["population"]
zip_data["freq_pop_75-inf"] = zip_data["pop_75-inf"] / zip_data["population"]

# Set negative median individual incomes to 0
zip_data["median_indiv_income"] = zip_data["median_indiv_income"].clip(lower=0)

This leaves us with the following features for the final dataset:

In [17]:
zip_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44336 entries, 0 to 44335
Data columns (total 30 columns):
zip                          44336 non-null object
city                         44336 non-null object
state                        44336 non-null object
latitude                     43689 non-null float64
longitude                    43689 non-null float64
population                   44336 non-null float64
pop_45-49                    44205 non-null float64
pop_50-54                    44205 non-null float64
pop_55-59                    44205 non-null float64
pop_60-64                    32674 non-null float64
pop_65-69                    32674 non-null float64
pop_70-74                    32674 non-null float64
pop_75-inf                   32674 non-null float64
median_indiv_income          32803 non-null float64
life_expectancy              36572 non-null float64
num_tax_returns_0-25k        29872 non-null float64
num_tax_returns_25k-50k      29872 non-null float64
num_tax_re

In [18]:
# Save everything
zip_data.to_csv("zip_data_final.csv")